In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.9.2


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
labels = ['Abstractism', 'Baroque', 'Byzantine', 'Cubism', 'Expressionism', 'High_Renaissance',
             'Impressionism', 'Mannerism', 'Muralism', 'Northern_Renaissance', 'Pop_Art',
             'Post-Impressionism', 'Primitivism', 'Proto_Renaissance', 'Realism', 'Romaticism',
             'Suprematism', 'Surrealism', 'Symbolism']

In [5]:
len(labels)

19

In [6]:
batch_size = 10


data_generator = ImageDataGenerator(validation_split=0.25,
                                   rescale=1.0/255.0,
                                   horizontal_flip=False,
                                   vertical_flip=False,
                                  )

train_data = data_generator.flow_from_directory(directory='/content/gdrive/My Drive/Data/genres/',
                                                    class_mode='categorical',
                                                    target_size=(224, 224),
                                                    batch_size=batch_size,
                                                    subset="training",
                                                    shuffle=True,
                                                    classes=labels
                                                   )

valid_data = data_generator.flow_from_directory(directory='/content/gdrive/My Drive/Data/genres/',
                                                    class_mode='categorical',
                                                    target_size=(224, 224),
                                                    batch_size=batch_size,
                                                    subset="validation",
                                                    shuffle=True,
                                                    classes=labels
                                                   )

STEP_PER_EPOCH_TRAIN = train_data.n//train_data.batch_size
STEP_PER_EPOCH_VALID = valid_data.n//valid_data.batch_size
print("Total number of batches =", STEP_PER_EPOCH_TRAIN, "and", STEP_PER_EPOCH_VALID)

Found 5805 images belonging to 19 classes.
Found 1925 images belonging to 19 classes.
Total number of batches = 580 and 192


Above we can see that we'll have 19 classes therefore our final Dense layer should be 19 to match it. Also we can see that using a batch size of 10 will give us 580 batches for the training data and 192 for the validation data. This is important because we will be using these numbers for the fitting.

In [ ]:
from tensorflow.keras import layers, models

CNN_model = models.Sequential()
CNN_model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
CNN_model.add(layers.MaxPooling2D((2, 2)))
CNN_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
CNN_model.add(layers.MaxPooling2D((2, 2)))
CNN_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
CNN_model.add(layers.Dropout(0.2))
CNN_model.add(layers.Flatten())
CNN_model.add(layers.Dense(19, activation='softmax'))

In [ ]:
CNN_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 64)        36928     
                                                                 
 dropout (Dropout)           (None, 52, 52, 64)        0

In [ ]:
CNN_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
CNN_model.fit(x=train_data, epochs=5, batch_size=10, steps_per_epoch=STEP_PER_EPOCH_TRAIN)

Epoch 1/5
580/580 [==============================] - 1351s 2s/step - loss: 2.3810 - accuracy: 0.2878
Epoch 2/5
580/580 [==============================] - 613s 1s/step - loss: 1.9634 - accuracy: 0.3903
Epoch 3/5
580/580 [==============================] - 616s 1s/step - loss: 1.3031 - accuracy: 0.5907
Epoch 4/5
580/580 [==============================] - 621s 1s/step - loss: 0.5278 - accuracy: 0.8361
Epoch 5/5
580/580 [==============================] - 615s 1s/step - loss: 0.1825 - accuracy: 0.9443


In [ ]:
CNN_model.evaluate(x=valid_data, batch_size=10, steps=STEP_PER_EPOCH_VALID)

192/192 [==============================] - 180s 940ms/step - loss: 7.3397 - accuracy: 0.1990


[7.339709758758545, 0.19895833730697632]

Testing a new model with higher dropout and increased convolutions.

In [7]:
from tensorflow.keras import layers, models

CNN_model = models.Sequential()
CNN_model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
CNN_model.add(layers.Dropout(0.2))
CNN_model.add(layers.MaxPooling2D((2, 2)))
CNN_model.add(layers.Conv2D(128, (3, 3), activation='relu'))
CNN_model.add(layers.Dropout(0.2))
CNN_model.add(layers.MaxPooling2D((2, 2)))
CNN_model.add(layers.Conv2D(128, (3, 3), activation='relu'))
CNN_model.add(layers.Dropout(0.2))
CNN_model.add(layers.MaxPooling2D((2, 2)))
CNN_model.add(layers.Conv2D(128, (3, 3), activation='relu'))
CNN_model.add(layers.Dropout(0.2))
CNN_model.add(layers.Flatten())
CNN_model.add(layers.Dense(19, activation='softmax'))

In [8]:
CNN_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
CNN_model.fit(x=train_data, epochs=5, batch_size=10, steps_per_epoch=STEP_PER_EPOCH_TRAIN, validation_data=valid_data)

Epoch 1/5
580/580 [==============================] - 2028s 3s/step - loss: 2.4005 - accuracy: 0.2733 - val_loss: 2.4432 - val_accuracy: 0.2504
Epoch 2/5
580/580 [==============================] - 1358s 2s/step - loss: 2.0879 - accuracy: 0.3474 - val_loss: 2.5089 - val_accuracy: 0.2145
Epoch 3/5
580/580 [==============================] - 1365s 2s/step - loss: 1.8085 - accuracy: 0.4311 - val_loss: 2.6303 - val_accuracy: 0.1829
Epoch 4/5
580/580 [==============================] - 1349s 2s/step - loss: 1.3820 - accuracy: 0.5674 - val_loss: 2.7678 - val_accuracy: 0.2104
Epoch 5/5
580/580 [==============================] - 1332s 2s/step - loss: 0.8767 - accuracy: 0.7198 - val_loss: 3.1544 - val_accuracy: 0.2021


Adding data augmentation and removing hidden layers to make the model smaller for the next test.

In [10]:
CNN_model = models.Sequential()
CNN_model.add(layers.RandomFlip("horizontal_and_vertical"))
CNN_model.add(layers.RandomRotation(0.3))
CNN_model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
CNN_model.add(layers.Dropout(0.2))
CNN_model.add(layers.MaxPooling2D((2, 2)))
CNN_model.add(layers.Flatten())
CNN_model.add(layers.Dense(19, activation='softmax'))
CNN_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
CNN_model.fit(x=train_data, epochs=5, batch_size=10, steps_per_epoch=STEP_PER_EPOCH_TRAIN, validation_data=valid_data)

Epoch 1/5
580/580 [==============================] - 458s 788ms/step - loss: 3.3557 - accuracy: 0.2468 - val_loss: 2.9343 - val_accuracy: 0.1574
Epoch 2/5
580/580 [==============================] - 454s 782ms/step - loss: 2.4379 - accuracy: 0.3034 - val_loss: 3.1350 - val_accuracy: 0.1709
Epoch 3/5
580/580 [==============================] - 452s 779ms/step - loss: 2.3282 - accuracy: 0.3175 - val_loss: 3.0188 - val_accuracy: 0.1958
Epoch 4/5
580/580 [==============================] - 458s 790ms/step - loss: 2.2845 - accuracy: 0.3199 - val_loss: 3.3072 - val_accuracy: 0.2073
Epoch 5/5
580/580 [==============================] - 475s 819ms/step - loss: 2.2535 - accuracy: 0.3232 - val_loss: 3.0733 - val_accuracy: 0.2099


It appears that the simpler model is performing better on the validation data so I will train it for an additional 3 epochs to see if it continues to improve.

In [12]:
CNN_model.fit(x=train_data, epochs=3, batch_size=10, steps_per_epoch=STEP_PER_EPOCH_TRAIN, validation_data=valid_data)

Epoch 1/3
580/580 [==============================] - 462s 796ms/step - loss: 2.2222 - accuracy: 0.3362 - val_loss: 3.5662 - val_accuracy: 0.2281
Epoch 2/3
580/580 [==============================] - 456s 786ms/step - loss: 2.1932 - accuracy: 0.3346 - val_loss: 3.7761 - val_accuracy: 0.1616
Epoch 3/3
580/580 [==============================] - 467s 805ms/step - loss: 2.1743 - accuracy: 0.3415 - val_loss: 3.1917 - val_accuracy: 0.1938
